In [1]:
import numpy as np
import pandas as pd

### 练习一

In [2]:
diamonds = pd.read_csv('../data/Diamonds.csv')

In [3]:
diamonds

,carat,color,depth,price
0,0.23,E,61.5,326
1,0.21,E,59.8,326
2,0.23,E,56.9,327
3,0.29,I,62.4,334
4,0.31,J,63.3,335
5,0.24,J,62.8,336
6,0.24,I,62.3,336
7,0.26,H,61.9,337
8,0.22,E,65.1,337
9,0.23,H,59.4,338


In [4]:
diamonds[diamonds.carat>1].price.max() - diamonds[diamonds.carat>1].price.min()

17561

In [5]:
diamonds.color.value_counts()

G    11292
E     9797
F     9542
H     8304
D     6775
I     5422
J     2808
Name: color, dtype: int64

In [6]:
diamonds.groupby(pd.cut(diamonds.depth,bins=diamonds.depth.quantile([0,.2, .4, .6, .8, 1]))).color.describe()

,count,unique,top,freq
depth,,,,
"(43.0, 60.8]",11294,7,E,2259
"(60.8, 61.6]",11831,7,G,2593
"(61.6, 62.1]",10403,7,G,2247
"(62.1, 62.7]",10137,7,G,2193
"(62.7, 79.0]",10273,7,G,2000


In [7]:
diamonds['price_per_carat'] = diamonds.price/diamonds.carat

In [8]:
diamonds['cuts'] = pd.cut(diamonds.depth,bins=diamonds.depth.quantile([0,.2, .4, .6, .8, 1]))

In [9]:
diamonds.groupby(['cuts', 'color'])['price_per_carat'].mean().groupby('cuts').idxmax()

cuts
(43.0, 60.8]    ((43.0, 60.8], I)
(60.8, 61.6]    ((60.8, 61.6], F)
(61.6, 62.1]    ((61.6, 62.1], F)
(62.1, 62.7]    ((62.1, 62.7], G)
(62.7, 79.0]    ((62.7, 79.0], G)
Name: price_per_carat, dtype: object

In [10]:
pd.cut(diamonds.carat, bins=[0, 0.5, 1, 1.5, 2,np.inf]).value_counts()

(0.0, 0.5]    18932
(0.5, 1.0]    17506
(1.0, 1.5]    12060
(1.5, 2.0]     3553
(2.0, inf]     1889
Name: carat, dtype: int64

In [11]:
carat_group = diamonds.groupby(pd.cut(diamonds.carat, bins=[0, 0.5, 1, 1.5, 2,float('inf')]))

In [12]:
# (carat_group.price.max()-carat_group.price.min()).sort_values(ascending=False)

In [13]:
# diamonds.groupby('color')['price','carat'].mean()

In [14]:
def f(nums):
    if not nums:        
        return 0
    res = 1                            
    cur_len = 1                        
    for i in range(1, len(nums)):      
        if nums[i-1] < nums[i]:        
            cur_len += 1                
            res = max(cur_len, res)     
        else:                       
            cur_len = 1                 
    return res

In [15]:
for name,group in carat_group:
    group = group.sort_values(by='depth')
    s = group['price']
    print(name,f(s.tolist()))

(0.0, 0.5] 8
(0.5, 1.0] 8
(1.0, 1.5] 7
(1.5, 2.0] 11
(2.0, inf] 7


In [16]:
for name,group in diamonds[['carat', 'price', 'color']].groupby('color'):
    L1 = np.array([np.ones(group.shape[0]), group['carat']]).reshape(2, group.shape[0])
    L2 = group['price']
    result = (np.linalg.inv(L1.dot(L1.T)).dot(L1)).dot(L2).reshape(2, 1)
    print('当颜色为%s时， 截距项为：%f，回归系数为：%f' % (name, result[0], result[1]))

当颜色为D时， 截距项为：-2361.017152，回归系数为：8408.353126
当颜色为E时， 截距项为：-2381.049600，回归系数为：8296.212783
当颜色为F时， 截距项为：-2665.806191，回归系数为：8676.658344
当颜色为G时， 截距项为：-2575.527643，回归系数为：8525.345779
当颜色为H时， 截距项为：-2460.418046，回归系数为：7619.098320
当颜色为I时， 截距项为：-2878.150356，回归系数为：7761.041169
当颜色为J时， 截距项为：-2920.603337，回归系数为：7094.192092


### 练习二

In [90]:
drugs = pd.read_csv('../data/Drugs.csv')

In [91]:
drugs

,YYYY,State,COUNTY,SubstanceName,DrugReports
0,2010,VA,ACCOMACK,Propoxyphene,1
1,2010,OH,ADAMS,Morphine,9
2,2010,PA,ADAMS,Methadone,2
3,2010,VA,ALEXANDRIA CITY,Heroin,5
4,2010,PA,ALLEGHENY,Hydromorphone,5
...,...,...,...,...,...
24057,2017,VA,WYTHE,Codeine,1
24058,2017,VA,WYTHE,Hydrocodone,19
24059,2017,VA,WYTHE,Tramadol,5
24060,2017,PA,YORK,ANPP,1


In [93]:
drugs.groupby(['State', 'COUNTY']).DrugReports.sum().sort_values(ascending=False)

State  COUNTY        
PA     PHILADELPHIA      50810
OH     HAMILTON          42933
PA     ALLEGHENY         33193
OH     CUYAHOGA          27630
       MONTGOMERY        21771
                         ...  
WV     PENDLETON             4
       LINCOLN               3
VA     NORTON CITY           2
WV     WIRT                  2
VA     COVINGTON CITY        1
Name: DrugReports, Length: 461, dtype: int64

In [94]:
drugs.groupby('State').DrugReports.sum().sort_values(ascending=False)

State
OH    246589
PA    184782
KY     82045
VA     73430
WV     23596
Name: DrugReports, dtype: int64

In [103]:
y_state_group = drugs[drugs.YYYY.isin(['2014','2015'])&(drugs.SubstanceName=='Heroin')].groupby(['YYYY','State']).sum()

In [110]:
y_state_group

DrugReports
YYYY State             
2014 KY            4362
     OH           20590
     PA           18454
     VA            3132
     WV            1516
2015 KY            4045
     OH           23347
     PA           18964
     VA            3583
     WV            1135

In [112]:
y_state_group.groupby('State').apply(lambda x:x.DrugReports[2015]-x.DrugReports[2014])

State  State
KY     KY       -317
OH     OH       2757
PA     PA        510
VA     VA        451
WV     WV       -381
Name: DrugReports, dtype: int64

In [107]:
y_sub_group = drugs[drugs.YYYY.isin(['2014','2015'])&(drugs.State=='OH')].groupby(['YYYY','SubstanceName']).sum()

In [114]:
y_sub_group

DrugReports
YYYY SubstanceName                        
2014 Acetyl fentanyl                    22
     Buprenorphine                    1629
     Codeine                           186
     Dextropropoxyphene                  1
     Dihydrocodeine                      1
     Fentanyl                         1353
     Heroin                          20590
     Hydrocodone                      1695
     Hydromorphone                      81
     Meperidine                          3
     Methadone                         285
     Mitragynine                         5
     Morphine                          368
     Oxycodone                        4242
     Oxymorphone                       126
     Pentazocine                         5
     Propoxyphene                       11
     Tramadol                          257
2015 ANPP                                1
     Acetyl fentanyl                   341
     Buprenorphine                    1821
     Butorphanol                         1
     Butyryl fentanyl                    8
     Codeine                           187
     Fentanyl                         3992
     Furanyl fentanyl                    1
     Heroin                          23347
     Hydrocodone                      1506
     Hydromorphone                      74
     Meperidine                          2
     Methadone                         249
     Mitragynine                         4
     Morphine                          423
     Oxycodone                        4360
     Oxymorphone                       157
     Pentazocine                         1
     Propoxyphene                       18
     Thebaine                            2
     Tramadol                          630
     p-Fluorobutyryl fentanyl            2

In [131]:
y_sub_group.groupby('SubstanceName').apply(lambda x:x.max()-x.min())

,DrugReports
SubstanceName,
ANPP,0
Acetyl fentanyl,319
Buprenorphine,192
Butorphanol,0
Butyryl fentanyl,0
Codeine,1
Dextropropoxyphene,0
Dihydrocodeine,0
Fentanyl,2639
